## Libraries

In [ ]:
# !pip install --user ipyvolume
# !pip install --user ipyvolume
# !pip install --user pythreejs
# !pip install --user PyntCloud
# !jupyter nbextension install --user --py pythreejs
# !jupyter nbextension enable --user --py pythreejs

In [ ]:
import sys
sys.path.insert(0, "..")
from ipywidgets import *
sys.path.insert(0, "..")
from IPython.display import clear_output
from pathlib import Path
from cgmcore import modelutils
from keras import models
import glob
import pickle
import matplotlib.pyplot as plt
import ipyvolume as ipv
from pyntcloud import PyntCloud
import glob2
import os
import pcl
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from random import randint
import re

## Dataset_path

Can be changed to visualize different set of point clouds

In [ ]:
#dataset_path = "/srv/data/etl/2018_10_31_14_19_42/SAM-SNG-087/1540793242066/pcd"

## Recursive search path and Timestamp

In [ ]:
def path_search(path):
    glob_search_path = os.path.join(path, "**/*.pcd")
    pcd_paths = glob2.glob(glob_search_path)
    pcd_paths = sorted(pcd_paths,key=lambda x: (os.path.splitext(x)[0]))
    print("pcd_paths", len(pcd_paths))
    return pcd_paths
    

In [ ]:
## User imput for selecting amount of days to be taken into account.
def period():

    import time

    N = int(input('Select how many days from now you want to check:'))

    current_milli_time = lambda: int(round(time.time() * 1000))

    ms = N * 24 *3600 *1000
    end = current_milli_time()    
    start = current_milli_time() - ms    
    return end,start
#To-do

## Model Selection
We can choose which model we want to use for the prediction in order to see the quality of our picture.
We are going to use the last trained pointnet model, as requires least preprocessing of the data.

In [ ]:
def get_model_path():
    all_paths = glob.glob("../../data/output/*")
    date_times = []
    for path in all_paths:
        split = path.split("/")[-1].split("-")
        date = split[0]
        time = split[1]
        date_time = date + "-" + time
        date_times.append(date_time)
    date_times = sorted(list(set(date_times)))
    date_times = date_times[-10:]
    #print(date_times)
    model_pointnet_path = []
    for date_time in date_times:
        # Load all models for date-time and take the latest pointnet
        model_paths = [path for path in all_paths if "model" in path and "pointnet" in path and date_time in path]
        if model_paths:
            # if we want more than one model, we should change this line using an append method instead
            model_pointnet_path = model_paths
    return model_pointnet_path

        # print(model_pointnet)    
    

In [ ]:
#alternative with load_weigths

def get_model():
    from cgmcore import modelutils
    dataset_parameters_pointclouds = {}
    dataset_parameters_pointclouds["pointcloud_target_size"] = 10000
    input_shape = (dataset_parameters_pointclouds["pointcloud_target_size"], 3)
    output_size = 1
    model_pointnet = modelutils.create_point_net(input_shape, output_size, hidden_sizes = [512, 256 ,128])
    #model_pointnet.summary()
    get_model_path()
    #         for model_pointnet in model_paths: #the function load_model is not working
#             
#             model = models.load_model(model_pointnet)
    model_pointnet.load_weights('../../data/output/20181101-2145-pointnet-model-weights.h5') 
    #load_weight gives also an error
    return model_pointnet



In [ ]:
model_pointnet = get_model()
model_pointnet.summary()       

## Complementary information to visualization

In [ ]:
def prediction(array_for_pred):
        indices = np.arange(0, array_for_pred.shape[0])
        indices = np.random.choice(indices, 10000)
        array_for_pred = array_for_pred[indices,0:3]
        prediction = model_pointnet.predict(array_for_pred.reshape(1, 10000, 3))
        return prediction
        
        
        
    

In [ ]:
def get_target(target_path):
    common_path= '/'.join(target_path.split("/")[:-2])
    # print(common_path + '/target.txt')
    
    real_value = np.loadtxt(common_path + '/target.txt')
    return real_value

In [ ]:
def interface(filepath, discarded, standing, laying):
    
    import os
    import shutil
#     valid = ['y','n']
#     valid2 = ['l','s']    
    print('Is the picture good?') 
    button_yes = widgets.Button(description="yes")
    display(button_yes)
    button_no = widgets.Button(description="no")
    display(button_no)
    button_standing = widgets.Button(description="standing")
    
    button_laying = widgets.Button(description="laying")
    
    def on_button_standing_clicked(b):
        print('Standing')
        if (os.path.isfile(standing)==True):
            print('This image was already there')
        else:
            shutil.copyfile(filepath, standing)
        button_standing.close()
        button_laying.close()
        #send to standing
        #shutil.copyfile(filePath, standing)
        clear_output()
        next(qc)
        
    def on_button_laying_clicked(b):
        print("Laying")
        if (os.path.isfile(laying)==True):
            print('This image was already there')
        else:
            shutil.copyfile(filepath, laying)
        button_standing.close()
        button_laying.close()
        #send to laying
        #shutil.copyfile(filePath, laying)
        clear_output()
        next(qc)
        
    def on_button_yes_clicked(b):
        print("Ok nice")
        button_yes.close()
        button_no.close()
        print('Is the child standing or laying?')
        display(button_standing)
        display(button_laying)
        
    def on_button_no_clicked(b):
        print("Ok we discard it")
        if (os.path.isfile(discarded)==True):
            print('This image was already there')
        else:
            shutil.copyfile(filepath, discarded)
        button_yes.close()
        button_no.close()
        #send to discard
        #shutil.copyfile(filepath, discarded)
        clear_output()
        next(qc)
        
    button_yes.on_click(on_button_yes_clicked)
    button_no.on_click(on_button_no_clicked)
    button_standing.on_click(on_button_standing_clicked)
    button_laying.on_click(on_button_laying_clicked)

    

## Manual Selection of Pictures
Ipyvolume visualization + Extra information

In [ ]:
def quality_check(directory):
    end,start = period()

    folders = os.listdir(directory)

    for name in folders:    
        subdir = os.path.join(directory,name)    
        r = os.listdir(subdir)
        for i in range(len(r)):
            pathh = os.path.join(subdir, r[i])
            finalpath = os.path.join(pathh, os.listdir(pathh)[1])

            pcd_paths = path_search(finalpath)
        
            for i in range(len(pcd_paths)):
                print(i)

                p = pcl.load(pcd_paths[i])

                parray = np.asarray(p)
                pred_value = prediction(parray)
                real_value = get_target(pcd_paths[i])
                head, tail = os.path.split(pcd_paths[i])
                #PATH HAS TO BE RENAMED, PLEASE CHOOSE THE RIGHT ONE: 
                standing = os.path.join('/home/jupyter-padronlv/cgm-ml/visualization/standing', tail)
                discarded = os.path.join('/home/jupyter-padronlv/cgm-ml/visualization/discarded', tail)
                laying = os.path.join('/home/jupyter-padronlv/cgm-ml/visualization/laying', tail)

                # print(parray[:,1],parray[:,0], parray[:,2])
                x,y,z = parray[:,1],parray[:,0], parray[:,2]
                ipv.quickscatter(x,y,z, marker="sphere", size = 0.5)
                ipv.style.use("dark")
                ipv.show()
                print('PREDICTED HEIGHT: {}, REAL HEIGHT: {}; ERROR: {}'.format(pred_value[0][0], real_value, np.abs(pred_value[0][0] - real_value)))
                interface(pcd_paths[i],discarded, standing, laying)
                yield

In [ ]:
directory = "/srv/data/etl/2018_10_31_14_19_42/"
qc = quality_check(directory)

In [ ]:
next(qc)